In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report

# Загрузка данных
data = pd.read_csv('healthcare_dataset.csv')

# Удаление столбца с именами пациентов (или других неинформативных для классификации данных)
data = data.drop('Name', axis=1)

# Предположим, что мы хотим предсказать тип медицинского состояния (Medical Condition)
# Разделение данных на признаки (X) и целевую переменную (y)
X = data.drop('Medical Condition', axis=1)
y = data['Medical Condition']

# Преобразование категориальных переменных с помощью One-Hot Encoding
X = pd.get_dummies(X)

# Разделение на обучающий и тестовый наборы данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создание и обучение первой модели CatBoostClassifier
model_1 = CatBoostClassifier(iterations=100, learning_rate=0.5, depth=3, loss_function='MultiClass')
model_1.fit(X_train, y_train)

# Оценка точности первой модели CatBoost на тестовом наборе данных
accuracy_1 = model_1.score(X_test, y_test)
print(f"Accuracy Model 1 (CatBoost): {accuracy_1}")

# Получение предсказаний первой модели CatBoost на тестовом наборе данных
predictions_1 = model_1.predict(X_test)
print(predictions_1)
# Вывод отчета о классификации для первой модели CatBoost
report_1 = classification_report(y_test, predictions_1)
print("Classification Report for Model 1 (CatBoost):")
print(report_1)

# Кросс-валидация для первой модели
cv_scores_1 = cross_val_score(model_1, X, y, cv=5, scoring='accuracy')
print(f"Mean Accuracy for Model 1 (CatBoost): {cv_scores_1.mean()}")

# Создание и обучение второй модели CatBoostClassifier с другими параметрами
model_2 = CatBoostClassifier(iterations=400, learning_rate=0.01, depth=8, loss_function='MultiClass')
model_2.fit(X_train, y_train)

# Оценка точности второй модели CatBoost на тестовом наборе данных
accuracy_2 = model_2.score(X_test, y_test)
print(f"Accuracy Model 2 (CatBoost): {accuracy_2}")

# Получение предсказаний второй модели CatBoost на тестовом наборе данных
predictions_2 = model_2.predict(X_test)
print(predictions_2)

# Вывод отчета о классификации для второй модели CatBoost
report_2 = classification_report(y_test, predictions_2)
print("Classification Report for Model 2 (CatBoost):")
print(report_2)

# Кросс-валидация для второй модели
cv_scores_2 = cross_val_score(model_2, X, y, cv=5, scoring='accuracy')
print(f"Mean Accuracy for Model 2 (CatBoost): {cv_scores_2.mean()}")


0:	learn: 1.7902985	total: 170ms	remaining: 16.8s
1:	learn: 1.7902036	total: 187ms	remaining: 9.17s
2:	learn: 1.7885977	total: 203ms	remaining: 6.57s
3:	learn: 1.7883159	total: 220ms	remaining: 5.28s
4:	learn: 1.7879283	total: 236ms	remaining: 4.49s
5:	learn: 1.7878432	total: 252ms	remaining: 3.96s
6:	learn: 1.7873825	total: 268ms	remaining: 3.57s
7:	learn: 1.7867476	total: 285ms	remaining: 3.27s
8:	learn: 1.7860773	total: 301ms	remaining: 3.05s
9:	learn: 1.7859282	total: 319ms	remaining: 2.87s
10:	learn: 1.7853994	total: 335ms	remaining: 2.71s
11:	learn: 1.7852619	total: 351ms	remaining: 2.58s
12:	learn: 1.7851328	total: 367ms	remaining: 2.46s
13:	learn: 1.7849921	total: 384ms	remaining: 2.36s
14:	learn: 1.7849034	total: 402ms	remaining: 2.28s
15:	learn: 1.7848058	total: 419ms	remaining: 2.2s
16:	learn: 1.7845899	total: 435ms	remaining: 2.12s
17:	learn: 1.7840457	total: 451ms	remaining: 2.06s
18:	learn: 1.7836814	total: 468ms	remaining: 2s
19:	learn: 1.7834464	total: 485ms	remaining: 

59:	learn: 1.7759692	total: 1.11s	remaining: 741ms
60:	learn: 1.7753252	total: 1.13s	remaining: 722ms
61:	learn: 1.7751877	total: 1.15s	remaining: 703ms
62:	learn: 1.7749810	total: 1.17s	remaining: 685ms
63:	learn: 1.7746592	total: 1.18s	remaining: 666ms
64:	learn: 1.7742856	total: 1.2s	remaining: 647ms
65:	learn: 1.7741257	total: 1.22s	remaining: 628ms
66:	learn: 1.7737651	total: 1.24s	remaining: 610ms
67:	learn: 1.7732814	total: 1.25s	remaining: 591ms
68:	learn: 1.7729840	total: 1.27s	remaining: 573ms
69:	learn: 1.7727199	total: 1.29s	remaining: 554ms
70:	learn: 1.7725674	total: 1.31s	remaining: 534ms
71:	learn: 1.7721446	total: 1.32s	remaining: 516ms
72:	learn: 1.7719703	total: 1.34s	remaining: 497ms
73:	learn: 1.7718749	total: 1.36s	remaining: 478ms
74:	learn: 1.7717841	total: 1.38s	remaining: 460ms
75:	learn: 1.7714396	total: 1.4s	remaining: 441ms
76:	learn: 1.7712888	total: 1.41s	remaining: 422ms
77:	learn: 1.7709299	total: 1.43s	remaining: 404ms
78:	learn: 1.7707158	total: 1.45s

26:	learn: 1.7838115	total: 454ms	remaining: 1.23s
27:	learn: 1.7837195	total: 470ms	remaining: 1.21s
28:	learn: 1.7833325	total: 487ms	remaining: 1.19s
29:	learn: 1.7830600	total: 503ms	remaining: 1.17s
30:	learn: 1.7827406	total: 519ms	remaining: 1.16s
31:	learn: 1.7822805	total: 538ms	remaining: 1.14s
32:	learn: 1.7820379	total: 555ms	remaining: 1.13s
33:	learn: 1.7814428	total: 573ms	remaining: 1.11s
34:	learn: 1.7811207	total: 590ms	remaining: 1.09s
35:	learn: 1.7806555	total: 608ms	remaining: 1.08s
36:	learn: 1.7802522	total: 625ms	remaining: 1.06s
37:	learn: 1.7800417	total: 642ms	remaining: 1.05s
38:	learn: 1.7793545	total: 659ms	remaining: 1.03s
39:	learn: 1.7790893	total: 676ms	remaining: 1.01s
40:	learn: 1.7788131	total: 694ms	remaining: 998ms
41:	learn: 1.7785392	total: 711ms	remaining: 982ms
42:	learn: 1.7782338	total: 728ms	remaining: 966ms
43:	learn: 1.7778322	total: 746ms	remaining: 950ms
44:	learn: 1.7776481	total: 765ms	remaining: 935ms
45:	learn: 1.7768760	total: 784

88:	learn: 1.7655855	total: 1.53s	remaining: 189ms
89:	learn: 1.7653948	total: 1.54s	remaining: 172ms
90:	learn: 1.7652620	total: 1.56s	remaining: 154ms
91:	learn: 1.7650550	total: 1.58s	remaining: 137ms
92:	learn: 1.7647241	total: 1.59s	remaining: 120ms
93:	learn: 1.7644964	total: 1.61s	remaining: 103ms
94:	learn: 1.7641672	total: 1.63s	remaining: 85.7ms
95:	learn: 1.7638177	total: 1.65s	remaining: 68.5ms
96:	learn: 1.7635059	total: 1.66s	remaining: 51.4ms
97:	learn: 1.7631642	total: 1.68s	remaining: 34.3ms
98:	learn: 1.7630702	total: 1.7s	remaining: 17.1ms
99:	learn: 1.7628717	total: 1.71s	remaining: 0us
0:	learn: 1.7897625	total: 18.3ms	remaining: 1.81s
1:	learn: 1.7892365	total: 35.6ms	remaining: 1.75s
2:	learn: 1.7891791	total: 53.3ms	remaining: 1.72s
3:	learn: 1.7887297	total: 70.8ms	remaining: 1.7s
4:	learn: 1.7886819	total: 87.6ms	remaining: 1.67s
5:	learn: 1.7885906	total: 105ms	remaining: 1.65s
6:	learn: 1.7883380	total: 122ms	remaining: 1.62s
7:	learn: 1.7867251	total: 140ms

48:	learn: 1.7873454	total: 12.4s	remaining: 1m 28s
49:	learn: 1.7873068	total: 12.6s	remaining: 1m 28s
50:	learn: 1.7872808	total: 12.9s	remaining: 1m 28s
51:	learn: 1.7872576	total: 13.1s	remaining: 1m 27s
52:	learn: 1.7872397	total: 13.4s	remaining: 1m 27s
53:	learn: 1.7872068	total: 13.6s	remaining: 1m 27s
54:	learn: 1.7871666	total: 13.9s	remaining: 1m 27s
55:	learn: 1.7871535	total: 14.1s	remaining: 1m 26s
56:	learn: 1.7871337	total: 14.4s	remaining: 1m 26s
57:	learn: 1.7868873	total: 14.6s	remaining: 1m 26s
58:	learn: 1.7868738	total: 14.9s	remaining: 1m 26s
59:	learn: 1.7867195	total: 15.1s	remaining: 1m 25s
60:	learn: 1.7866585	total: 15.4s	remaining: 1m 25s
61:	learn: 1.7866142	total: 15.6s	remaining: 1m 25s
62:	learn: 1.7864791	total: 15.9s	remaining: 1m 25s
63:	learn: 1.7859759	total: 16.2s	remaining: 1m 24s
64:	learn: 1.7859636	total: 16.4s	remaining: 1m 24s
65:	learn: 1.7859415	total: 16.7s	remaining: 1m 24s
66:	learn: 1.7859098	total: 16.9s	remaining: 1m 23s
67:	learn: 1

...

168:	learn: 1.7721966	total: 58.8s	remaining: 1m 20s
169:	learn: 1.7721813	total: 59.1s	remaining: 1m 19s
170:	learn: 1.7721286	total: 59.5s	remaining: 1m 19s
171:	learn: 1.7721116	total: 59.8s	remaining: 1m 19s
172:	learn: 1.7719869	total: 1m	remaining: 1m 19s
173:	learn: 1.7719812	total: 1m	remaining: 1m 18s
174:	learn: 1.7719271	total: 1m	remaining: 1m 18s
175:	learn: 1.7719202	total: 1m 1s	remaining: 1m 17s
176:	learn: 1.7719077	total: 1m 1s	remaining: 1m 17s
177:	learn: 1.7718871	total: 1m 1s	remaining: 1m 17s
178:	learn: 1.7716742	total: 1m 2s	remaining: 1m 16s
179:	learn: 1.7715890	total: 1m 2s	remaining: 1m 16s
180:	learn: 1.7713307	total: 1m 2s	remaining: 1m 16s
181:	learn: 1.7711670	total: 1m 3s	remaining: 1m 15s
182:	learn: 1.7711612	total: 1m 3s	remaining: 1m 15s
183:	learn: 1.7711171	total: 1m 4s	remaining: 1m 15s
184:	learn: 1.7710436	total: 1m 4s	remaining: 1m 14s
185:	learn: 1.7710330	total: 1m 4s	remaining: 1m 14s
186:	learn: 1.7710208	total: 1m 5s	remaining: 1m 14s
18

324:	learn: 1.7593959	total: 1m 53s	remaining: 26.2s
325:	learn: 1.7589600	total: 1m 53s	remaining: 25.8s
326:	learn: 1.7589535	total: 1m 54s	remaining: 25.5s
327:	learn: 1.7589434	total: 1m 54s	remaining: 25.1s
328:	learn: 1.7589232	total: 1m 54s	remaining: 24.8s
329:	learn: 1.7587346	total: 1m 55s	remaining: 24.4s
330:	learn: 1.7582084	total: 1m 55s	remaining: 24.1s
331:	learn: 1.7582019	total: 1m 55s	remaining: 23.7s
332:	learn: 1.7581902	total: 1m 56s	remaining: 23.4s
333:	learn: 1.7581481	total: 1m 56s	remaining: 23s
334:	learn: 1.7581408	total: 1m 56s	remaining: 22.7s
335:	learn: 1.7577442	total: 1m 57s	remaining: 22.3s
336:	learn: 1.7577414	total: 1m 57s	remaining: 22s
337:	learn: 1.7577254	total: 1m 57s	remaining: 21.6s
338:	learn: 1.7573383	total: 1m 58s	remaining: 21.3s
339:	learn: 1.7573014	total: 1m 58s	remaining: 20.9s
340:	learn: 1.7572852	total: 1m 58s	remaining: 20.6s
341:	learn: 1.7572328	total: 1m 59s	remaining: 20.2s
342:	learn: 1.7571477	total: 1m 59s	remaining: 19.

83:	learn: 1.7847070	total: 29.5s	remaining: 1m 50s
84:	learn: 1.7846952	total: 29.8s	remaining: 1m 50s
85:	learn: 1.7846761	total: 30.2s	remaining: 1m 50s
86:	learn: 1.7846569	total: 30.6s	remaining: 1m 49s
87:	learn: 1.7846158	total: 30.9s	remaining: 1m 49s
88:	learn: 1.7845985	total: 31.3s	remaining: 1m 49s
89:	learn: 1.7845922	total: 31.6s	remaining: 1m 48s
90:	learn: 1.7839742	total: 32s	remaining: 1m 48s
91:	learn: 1.7839336	total: 32.3s	remaining: 1m 48s
92:	learn: 1.7839038	total: 32.7s	remaining: 1m 47s
93:	learn: 1.7838790	total: 33s	remaining: 1m 47s
94:	learn: 1.7837194	total: 33.4s	remaining: 1m 47s
95:	learn: 1.7837076	total: 33.7s	remaining: 1m 46s
96:	learn: 1.7836818	total: 34.1s	remaining: 1m 46s
97:	learn: 1.7836516	total: 34.5s	remaining: 1m 46s
98:	learn: 1.7836360	total: 34.8s	remaining: 1m 45s
99:	learn: 1.7835141	total: 35.2s	remaining: 1m 45s
100:	learn: 1.7834538	total: 35.5s	remaining: 1m 45s
101:	learn: 1.7834283	total: 35.9s	remaining: 1m 44s
102:	learn: 1.

239:	learn: 1.7724810	total: 1m 24s	remaining: 56.1s
240:	learn: 1.7724474	total: 1m 24s	remaining: 55.7s
241:	learn: 1.7721898	total: 1m 24s	remaining: 55.4s
242:	learn: 1.7720660	total: 1m 25s	remaining: 55s
243:	learn: 1.7720557	total: 1m 25s	remaining: 54.7s
244:	learn: 1.7720513	total: 1m 25s	remaining: 54.3s
245:	learn: 1.7720000	total: 1m 26s	remaining: 54s
246:	learn: 1.7719906	total: 1m 26s	remaining: 53.7s
247:	learn: 1.7719100	total: 1m 26s	remaining: 53.3s
248:	learn: 1.7718547	total: 1m 27s	remaining: 53s
249:	learn: 1.7718333	total: 1m 27s	remaining: 52.6s
250:	learn: 1.7718155	total: 1m 28s	remaining: 52.3s
251:	learn: 1.7717863	total: 1m 28s	remaining: 51.9s
252:	learn: 1.7711439	total: 1m 28s	remaining: 51.6s
253:	learn: 1.7711339	total: 1m 29s	remaining: 51.2s
254:	learn: 1.7711111	total: 1m 29s	remaining: 50.9s
255:	learn: 1.7711050	total: 1m 29s	remaining: 50.5s
256:	learn: 1.7710880	total: 1m 30s	remaining: 50.2s
257:	learn: 1.7709832	total: 1m 30s	remaining: 49.8s

395:	learn: 1.7604443	total: 2m 19s	remaining: 1.41s
396:	learn: 1.7603118	total: 2m 20s	remaining: 1.06s
397:	learn: 1.7603065	total: 2m 20s	remaining: 705ms
398:	learn: 1.7602882	total: 2m 20s	remaining: 352ms
399:	learn: 1.7602701	total: 2m 20s	remaining: 0us
Mean Accuracy for Model 2 (CatBoost): 0.1592


Программа реализует загрузку и выбранного мною датасета,предварительно удалю столбец "Name" так как он не влияет никак на значения,разделяю данные на признаки,так как в датаесети имеются категоральные типы данных кодируем их с помощью One-Hot Encoding,разделению на обучающий и тестовый набор данных в соотношении 80/20,создаю 2 модели CatBoostClassifier (с разными гиперпараметрами), чтобы можно было проанализировать изменения в результате.Для каждой модели измеряется точность и выводится classification_report,также проводится кросс-валидация в 5 фолдов,её тоже вывел со средним значением.